In [211]:
import torch
import numpy as np
from glob import glob
import os
import matplotlib
PATH = '/home/michael/Dropbox/Master thesis/results_graph_dc_new_lr_0_5/2_out/simple/club_feature_analysis/FeatureAnalysisGraph-DC_2_out_only_CLUB_Diff_global_private_only_2_branches_NEW_sim_loss_lr_0_0_A0_F0_0005_F0_0005_H10/FedDomSep_GraphDC_gin_on_fs_contest_data_lr0.0_lstep1_/features'

train_test_val = 'val'

save_path = '/home/michael/Dropbox/Master thesis/results_graph_dc_new_lr_0_5/2_out/simple/club_feature_analysis/feature_analysis/diff_0_0005' +'/'+ train_test_val + '/'
if train_test_val == 'train':
    nun_samples = 11649
elif train_test_val == 'val':
    nun_samples = 3883
elif train_test_val == 'test':
    nun_samples = 3889

In [212]:
font = {'size'   : 22}

matplotlib.rc('font', **font)

In [213]:
#interm_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
global_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
local_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
labels = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}


In [214]:
all_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, '*.pt'))]
all_files.sort()

In [215]:
def get_files(path):
    for client in range(1, 13+1):
        client_path = f"{PATH}/client_{client}"
        all_files = [file
                 for path, subdir, files in os.walk(client_path)
                 for file in glob(os.path.join(path, '*.pt'))]
        all_files.sort()
        for file in all_files:
            file_name = os.path.basename(file)
            file_name_without_ending = file_name[:-3]
            if file_name_without_ending.startswith('x_global_' + train_test_val) and \
                    not file_name_without_ending.endswith('labels'):
                global_trains[client].append(torch.load(file).cpu().detach().numpy())
            if file_name_without_ending.startswith('x_local_' + train_test_val) and not\
                    file_name_without_ending.endswith('labels'):
                local_trains[client].append(torch.load(file).cpu().detach().numpy())
            if file_name_without_ending.startswith(train_test_val) and file_name_without_ending.endswith('labels'):
                labels[client].append(torch.load(file).cpu().detach().numpy())
        #interm_trains[client] = np.concatenate(interm_trains[client])
        global_trains[client] = np.concatenate(global_trains[client])
        local_trains[client] = np.concatenate(local_trains[client])
        labels[client] = np.concatenate(labels[client])


In [216]:
get_files(PATH)

In [217]:
import numpy as np
#concat_X=global_train
data_out_local_global = {}
data_out_local = {}
data_out_global = {}

for key in local_trains.keys():
    data_out_local_global[key] = global_trains[key] + local_trains[key]
    data_out_local[key] = local_trains[key]
    data_out_global[key] = global_trains[key]

X_concat_local_global = []
X_concat_local = []
X_concat_global = []
labels_concat = []

for key in local_trains.keys():
    X_concat_local_global.append(data_out_local_global[key])
    X_concat_local.append(data_out_local[key])
    X_concat_global.append(data_out_global[key])
    labels_concat.append(labels[key])

X_concat_local_global = np.concatenate(X_concat_local_global)
X_concat_local = np.concatenate(X_concat_local)
X_concat_global = np.concatenate(X_concat_global)

labels_concat = np.concatenate(labels_concat)

In [218]:
from sklearn.manifold import TSNE

In [219]:
labels_global = labels.copy()


In [220]:
X_concat = np.concatenate([X_concat_local_global, X_concat_local, X_concat_global])
labels_double_concat = np.concatenate([labels_concat, labels_concat, labels_concat])

In [221]:
# Feature Analysis for client 9:
global_trains[9]

array([[0.        , 0.03985845, 0.03987687, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11795227],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.01642824],
       [0.        , 0.        , 0.        , ..., 0.        , 0.05674228,
        0.01586871],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.04666327]], dtype=float32)

In [222]:
local_trains[9]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03631042, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.20424464],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11705239],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05572657, ..., 0.        , 0.        ,
        0.01905365]], dtype=float32)

In [223]:
cos_sim = torch.nn.CosineSimilarity()

In [224]:
isExist = os.path.exists(save_path)
if not isExist:
    os.makedirs(save_path)

In [225]:
import pandas as pd

df = pd.DataFrame()

In [226]:
num_total_features = []
num_local_features_not_0 = []
avg_local_features_not_0 = []
num_global_features_not_0 = []
num_features_global_local = []
avg_global_features_not_0 = []
num_local_global_features_not_0 = []
avg_local_global_features_not_0 = []
cos_sim_local_global = []
cos_sim_local_local_global = []
cos_sim_global_local_global = []

In [227]:
for client in range(1, 13+1):


    local_train_curr = local_trains[client]
    global_train_curr = global_trains[client]

    num_total_features.append(local_train_curr.shape[0] * local_train_curr.shape[1])
    num_total_features_curr = local_train_curr.shape[0] * local_train_curr.shape[1]

    local_global = local_train_curr + global_train_curr

    mult_local_global = global_train_curr*local_train_curr

    num_local_features_not_0.append(np.sum(local_train_curr != 0) / num_total_features_curr)

    avg_local_features_not_0.append(local_train_curr.sum() / np.sum(local_train_curr != 0))
    num_global_features_not_0.append(np.sum(global_train_curr != 0) / num_total_features_curr)
    avg_global_features_not_0.append(global_train_curr.sum() / np.sum(global_train_curr != 0))
    num_local_global_features_not_0.append(np.sum(local_global != 0) / num_total_features_curr)
    avg_local_global_features_not_0.append(local_global.sum() / np.sum(local_global != 0))

    num_features_global_local.append(np.sum(mult_local_global != 0) / num_total_features_curr)

    cos_sim_local_global.append(cos_sim(torch.Tensor(global_train_curr), torch.Tensor(local_train_curr)).mean().item())
    cos_sim_local_local_global.append(cos_sim(torch.Tensor(local_train_curr), torch.Tensor(local_global)).mean().item())
    cos_sim_global_local_global.append(cos_sim(torch.Tensor(global_train_curr), torch.Tensor(local_global)).mean().item())
    """
    print(f"############ Client {client} ##############")
    print(f"total features: {round(num_total_features,2)}")
    print(f"num_local_features_not_0:\n{round(num_local_features_not_0,2)}")
    print(f"avg_local_features_not_0:\n{round(avg_local_features_not_0,2)}")

    print(f"num_global_features_not_0:\n{round(num_global_features_not_0,2)}")
    print(f"num_features_global_local:\n{round(num_features_global_local,2)}")
    print(f"avg_global_features_not_0:\n{round(avg_global_features_not_0,2)}")

    print(f"num_local_global_features_not_0:\n{round(num_local_global_features_not_0,2)}")
    print(f"avg_local_global_features_not_0:\n{round(avg_local_global_features_not_0,2)}")

    print(f"cos_sim_global_local:\n{round(cos_sim_local_global,2)}")
    print(f"cos_sim_local_local_global:\n{round(cos_sim_local_local_global,2)}")
    print(f"cos_sim_global_local_global:\n{round(cos_sim_global_local_global,2)}")

    """

In [228]:
data_map = {
    ('total','# Features'): num_total_features,
    ('local', '# features != 0'): num_local_features_not_0,
    ('local', 'avg features != 0'): avg_local_features_not_0,
    ('global', '# features != 0'): num_global_features_not_0,
    ('global', '# Features != 0 in both local and global'): num_features_global_local,
    ('global', 'avg features != 0'): avg_global_features_not_0,
    ('local-global', '# features != 0'): num_local_global_features_not_0,
    ('local-global', 'avg features != 0'): avg_local_global_features_not_0,
    ('cos_sim', 'local_global',): cos_sim_local_global,
    ('cos_sim', 'local__local-global',): cos_sim_local_local_global,
    ('cos_sim', 'global__local-global',): cos_sim_global_local_global,
}
df = pd.DataFrame(data=data_map)
df.to_csv(save_path + 'statistics.csv')

In [229]:
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random',
                  perplexity=25).fit_transform(X_concat, labels_double_concat)
X_embedded.shape[0] // 2

5824

In [230]:
X_embedded_local_global = X_embedded[:nun_samples,:]
X_embedded_local = X_embedded[nun_samples:nun_samples+nun_samples,:]
X_embedded_global = X_embedded[nun_samples+nun_samples:,:]


In [231]:
X_embedded.shape[0]/3

3883.0

In [232]:
#X_embedded_global = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=25).fit_transform(X_concat_global, labels_concat)
#X_embedded_local = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=25).fit_transform(X_concat_interm, labels_concat)
#X_embedded_interm = X_embedded_local

In [233]:
#X_embedded_interm = X_embedded_local

In [234]:
X_embedded_global_clients = {}
X_embedded_local_clients = {}
X_embedded_local_global_clients = {}
labels_clients = {}
total_size = 0

for client in range(1, 13+1):
    print(f"size client {client}: {local_trains[client].shape[0]}")
    size = local_trains[client].shape[0]
    size_old = total_size
    total_size += size
    X_embedded_global_clients[client] = X_embedded_global[size_old:total_size,:]
    X_embedded_local_clients[client] = X_embedded_local[size_old:total_size,:]
    X_embedded_local_global_clients[client] = X_embedded_local_global[size_old:total_size,:]
    labels_clients[client] = labels_concat[size_old:total_size]

size client 1: 38
size client 2: 81
size client 3: 93
size client 4: 151
size client 5: 69
size client 6: 400
size client 7: 822
size client 8: 867
size client 9: 825
size client 10: 67
size client 11: 70
size client 12: 200
size client 13: 200


In [235]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30, 30))
plt.rcParams["figure.figsize"] = (30,30)
plt.rc('xtick', labelsize=64)    # fontsize of the tick labels
plt.rc('ytick', labelsize=64)    # fontsize of the tick labels
plt.rc('legend', fontsize=64)
plt.rc('font', size=64)
plt.rcParams["axes.linewidth"] = 4

<Figure size 3000x3000 with 0 Axes>

In [236]:
"""yellow: label=1"""

'yellow: label=1'

In [237]:
client_num = 13

In [238]:
save_path

'/home/michael/Dropbox/Master thesis/results_graph_dc_new_lr_0_5/2_out/simple/club_feature_analysis/feature_analysis/diff_0_0005/val/'

In [239]:
colors=['red', 'blue']
for client in range(1, client_num+1):
    for i in range(1+1):
        labels_use = labels_clients[client] == i
        X_use = X_embedded_global_clients[client][labels_use, :]
        plt.scatter(X_use[:, 0], X_use[:, 1], c=[colors[i]]*X_use.shape[0], label=f'Label = {i}', s=150, )
    plt.xlim(-120, 120)
    plt.ylim(-120, 120)
    plt.legend(prop={'size': 60}, markerscale=4)
    frame1 = plt.gca()
    frame1.axes.xaxis.set_ticklabels([])
    frame1.axes.yaxis.set_ticklabels([])
    plt.savefig(save_path + 'client_' + str(client) + '_global')
    plt.clf()

    for i in range(1+1):
        labels_use = labels_clients[client] == i
        X_use = X_embedded_local_clients[client][labels_use, :]
        plt.scatter(X_use[:, 0], X_use[:, 1], c=[colors[i]]*X_use.shape[0], label=f'Label = {i}', s=150, )
    plt.xlim(-120, 120)
    plt.ylim(-120, 120)
    plt.legend(prop={'size': 60}, markerscale=4)
    frame1 = plt.gca()
    frame1.axes.xaxis.set_ticklabels([])
    frame1.axes.yaxis.set_ticklabels([])
    plt.savefig(save_path + 'client_' + str(client) + '_local')
    plt.clf()

    for i in range(1+1):
        labels_use = labels_clients[client] == i
        X_use = X_embedded_local_global_clients[client][labels_use, :]
        plt.scatter(X_use[:, 0], X_use[:, 1], c=[colors[i]]*X_use.shape[0], label=f'Label = {i}', s=150)
    plt.xlim(-120, 120)
    plt.ylim(-120, 120)
    plt.legend(prop={'size': 60}, markerscale=4)
    frame1 = plt.gca()
    frame1.axes.xaxis.set_ticklabels([])
    frame1.axes.yaxis.set_ticklabels([])
    plt.savefig(save_path + 'client_' + str(client) + '_local_global', bbox_inches='tight')
    plt.clf()




<Figure size 3000x3000 with 0 Axes>

In [240]:
plt.scatter(X_embedded_global_clients[client_num][:, 0], X_embedded_global_clients[client_num][:, 1], c=labels_clients[client_num], s=150 )
plt.xlim(-120, 120)
plt.ylim(-120, 120)
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [241]:
"""
plt.scatter(X_embedded_global_clients[client_num][:, 0], X_embedded_global_clients[client_num][:, 1], c=labels_clients[client_num], )
plt.xlim(-110, 110)
plt.ylim(-100, 90)
plt.show()
"""

'\nplt.scatter(X_embedded_global_clients[client_num][:, 0], X_embedded_global_clients[client_num][:, 1], c=labels_clients[client_num], )\nplt.xlim(-110, 110)\nplt.ylim(-100, 90)\nplt.show()\n'

In [242]:
"""
plt.scatter(X_embedded_local_clients[client_num][:, 0], X_embedded_local_clients[client_num][:, 1], c=labels_clients[client_num], )
plt.xlim(-110, 110)
plt.ylim(-100, 90)
plt.show()
"""

'\nplt.scatter(X_embedded_local_clients[client_num][:, 0], X_embedded_local_clients[client_num][:, 1], c=labels_clients[client_num], )\nplt.xlim(-110, 110)\nplt.ylim(-100, 90)\nplt.show()\n'

In [243]:
"""
plt.scatter(X_embedded_local_global_clients[client_num][:, 0], X_embedded_local_global_clients[client_num][:, 1], c=labels_clients[client_num], )
plt.xlim(-110, 110)
plt.ylim(-100, 90)
plt.show()
"""

'\nplt.scatter(X_embedded_local_global_clients[client_num][:, 0], X_embedded_local_global_clients[client_num][:, 1], c=labels_clients[client_num], )\nplt.xlim(-110, 110)\nplt.ylim(-100, 90)\nplt.show()\n'

In [244]:
#import matplotlib.pyplot as plt
#fig = plt.figure(figsize=(30, 18))
#plt.rcParams["figure.figsize"] = (30,18)

In [245]:
colors=['red', 'blue']
for i in range(1+1):
    labels_use = labels_concat == i
    X_use = X_embedded_global[labels_use, :]
    plt.scatter(X_use[:, 0], X_use[:, 1], c=[colors[i]]*X_use.shape[0], label=f'Label = {i}', s=150)
plt.legend(prop={'size': 60}, markerscale=4)
plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'all_clients' + '_global', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [246]:

colors=['red', 'blue']
for i in range(1+1):
    labels_use = labels_concat == i
    X_use = X_embedded_local[labels_use, :]
    plt.scatter(X_use[:, 0], X_use[:, 1], c=[colors[i]]*X_use.shape[0], label=f'Label = {i}', s=150)
plt.legend(prop={'size': 60}, markerscale=4)
plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'all_clients' + '_local', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [247]:
colors=['red', 'blue']
for i in range(1+1):
    labels_use = labels_concat == i
    X_use = X_embedded_local_global[labels_use, :]
    plt.scatter(X_use[:, 0], X_use[:, 1], c=[colors[i]]*X_use.shape[0], label=f'Label = {i}', s=150)
plt.legend(prop={'size': 60}, markerscale=4)
plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'all_clients' + '_local_global', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [248]:

colors = []
NUM_COLORS = 13
cm = plt.get_cmap('tab20')
colors_use = list(map(cm, colors))
fig = plt.figure()
for client in range(1, client_num+1):
    data_size = X_embedded_global_clients[client].shape[0]
    colors_curr = data_size*[client]
    colors += colors_curr

ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for client in range(1, client_num+1):
    labels_0 = labels_clients[client] == 0
    X_embedded_global_clients_only_label_0 = X_embedded_global_clients[client][labels_0, :]
    ax.scatter(X_embedded_global_clients_only_label_0[:,0],
               X_embedded_global_clients_only_label_0[:,1], label = f'{client}', s=150)



#ax.legend_elements()
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.05,
                 box.width, box.height * 0.95])
ax.legend(title="Client:", prop={'size': 60}, markerscale=4, loc='upper center', bbox_to_anchor=(0.5,
    -0.01), ncol=13)

plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'global_comparison_label_0', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [249]:

colors = []
NUM_COLORS = 13
cm = plt.get_cmap('tab20')
colors_use = list(map(cm, colors))
fig = plt.figure()
for client in range(1, client_num+1):
    data_size = X_embedded_global_clients[client].shape[0]
    colors_curr = data_size*[client]
    colors += colors_curr

ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for client in range(1, client_num+1):
    labels_1 = labels_clients[client] == 1
    X_embedded_global_clients_only_label_1 = X_embedded_global_clients[client][labels_1, :]
    ax.scatter(X_embedded_global_clients_only_label_1[:,0],
               X_embedded_global_clients_only_label_1[:,1], label = f'{client}', s=150)



#ax.legend_elements()
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.05,
                 box.width, box.height * 0.95])
ax.legend(title="Client:", prop={'size': 60}, markerscale=4, loc='upper center', bbox_to_anchor=(0.5,
    -0.01), ncol=13)

plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'global_comparison_label_1', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [250]:

colors = []
NUM_COLORS = 13
cm = plt.get_cmap('tab20')
colors_use = list(map(cm, colors))
fig = plt.figure()
for client in range(1, client_num+1):
    data_size = X_embedded_local_clients[client].shape[0]
    colors_curr = data_size*[client]
    colors += colors_curr

ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for client in range(1, client_num+1):
    labels_0 = labels_clients[client] == 0
    X_embedded_local_clients_only_label_0 = X_embedded_local_clients[client][labels_0, :]
    ax.scatter(X_embedded_local_clients_only_label_0[:,0],
               X_embedded_local_clients_only_label_0[:,1], label = f'{client}', s=150)



#ax.legend_elements()
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.05,
                 box.width, box.height * 0.95])
ax.legend(title="Client:", prop={'size': 60}, markerscale=4, loc='upper center', bbox_to_anchor=(0.5,
    -0.01), ncol=13)

plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'local_comparison_label_0', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [251]:

colors = []
NUM_COLORS = 13
cm = plt.get_cmap('tab20')
colors_use = list(map(cm, colors))
fig = plt.figure()
for client in range(1, client_num+1):
    data_size = X_embedded_local_clients[client].shape[0]
    colors_curr = data_size*[client]
    colors += colors_curr

ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for client in range(1, client_num+1):
    labels_0 = labels_clients[client] == 1
    X_embedded_local_clients_only_label_0 = X_embedded_local_clients[client][labels_0, :]
    ax.scatter(X_embedded_local_clients_only_label_0[:,0],
               X_embedded_local_clients_only_label_0[:,1], label = f'{client}', s=150)


#ax.legend(title="Client:", loc='upper center', bbox_to_anchor=(0.5, -0.05),
#          fancybox=True, shadow=True, ncol=13)
#ax.legend_elements()
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.05,
                 box.width, box.height * 0.95])
ax.legend(title="Client:", prop={'size': 60}, markerscale=4, loc='upper center', bbox_to_anchor=(0.5,
    -0.01), ncol=13)
plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'local_comparison_label_1', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [252]:

colors = []
NUM_COLORS = 13
cm = plt.get_cmap('tab20')
colors_use = list(map(cm, colors))
fig = plt.figure()
for client in range(1, client_num+1):
    data_size = X_embedded_local_global_clients[client].shape[0]
    colors_curr = data_size*[client]
    colors += colors_curr

ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for client in range(1, client_num+1):
    labels_0 = labels_clients[client] == 0
    X_embedded_local_global_clients_only_label_0 = X_embedded_local_global_clients[client][labels_0, :]
    ax.scatter(X_embedded_local_global_clients_only_label_0[:,0],
               X_embedded_local_global_clients_only_label_0[:,1], label = f'{client}', s=150)


frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
#ax.legend_elements()
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.05,
                 box.width, box.height * 0.95])
ax.legend(title="Client:", prop={'size': 60}, markerscale=4, loc='upper center', bbox_to_anchor=(0.5,
    -0.01), ncol=13)

plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'local_global_comparison_label_0', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [253]:

colors = []
NUM_COLORS = 13
cm = plt.get_cmap('tab20')
colors_use = list(map(cm, colors))
fig = plt.figure()
for client in range(1, client_num+1):
    data_size = X_embedded_local_global_clients[client].shape[0]
    colors_curr = data_size*[client]
    colors += colors_curr

ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

for client in range(1, client_num+1):
    labels_0 = labels_clients[client] == 1
    X_embedded_local_global_clients_only_label_0 = X_embedded_local_global_clients[client][labels_0, :]
    ax.scatter(X_embedded_local_global_clients_only_label_0[:,0], X_embedded_local_global_clients_only_label_0[:,1], label = f'{client}', s=150)



#ax.legend_elements()
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.05,
                 box.width, box.height * 0.95])
ax.legend(title="Client:", prop={'size': 60}, markerscale=4, loc='upper center', bbox_to_anchor=(0.5,
    -0.01), ncol=13)

plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'local_global_comparison_label_1', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [254]:
colors=['red', 'blue']

labels_use = labels_concat == 0
X_use_local = X_embedded_local[labels_use, :]
X_use_global = X_embedded_global[labels_use, :]
plt.scatter(X_use_local[:, 0], X_use_local[:, 1], c=[colors[0]]*X_use_local.shape[0],
            label=f'Local', s=150)
plt.scatter(X_use_global[:, 0], X_use_global[:, 1], c=[colors[1]]*X_use_global.shape[0],
            label=f'Global', s=150)
plt.legend(title="Client:", prop={'size': 60}, markerscale=4)
plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])
plt.savefig(save_path + 'all_clients' + 'comparison_local_to_global_label_0', bbox_inches='tight')
plt.clf()

<Figure size 3000x3000 with 0 Axes>

In [255]:
colors=['red', 'blue']

labels_use = labels_concat == 1
X_use_local = X_embedded_local[labels_use, :]
X_use_global = X_embedded_global[labels_use, :]
plt.scatter(X_use_local[:, 0], X_use_local[:, 1], c=[colors[0]]*X_use_local.shape[0],
            label=f'Local', s=150)
plt.scatter(X_use_global[:, 0], X_use_global[:, 1], c=[colors[1]]*X_use_global.shape[0],
            label=f'Global', s=150)
plt.legend(title="Client:", prop={'size': 60}, markerscale=4)
plt.xlim(-120, 120)
plt.ylim(-120, 120)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])

plt.savefig(save_path + 'all_clients' + 'comparison_local_to_global_label_1', bbox_inches='tight')
plt.clf()


<Figure size 3000x3000 with 0 Axes>